In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# from fastai import *
# from fastai.tabular import *
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import gc
import os
import re
from sklearn.decomposition import PCA

In [0]:
print(os.listdir('../content/drive/My Drive/Colab Notebooks/'))

['calendar.csv', 'sales_train_validation.csv', 'sample_submission.csv', 'sell_prices.csv', 'kernel64a066fb59 (1) (1) (1) (1).ipynb', 'kernel64a066fb59 (1) (1) (1) (1) (1).ipynb', 'kernel64a066fb59 (1).ipynb', 'kernel64a066fb59 (1) (1) (1) (1) (2).ipynb', 'kernel64a066fb59 (1) (1) (1) (1) (2) (1).ipynb', 'kernel64a066fb59_(1)_(1)_(1)_(1)_(2)_(1) (1).ipynb', 'kernel64a066fb59_(1)_(1)_(1)_(1)_(2)_(1)_(1).ipynb', 'data_nn_shift10', 'data_NN_shift10_1', 'data_nn_shift10_2', '.ipynb_checkpoints', 'data_nn_shift28', 'Liverpool Ion', 'wavenet-with-shifted-rfc-proba-and-cbr.ipynb', 'Liverpool_Ion_Wavenet_(1) (1).ipynb', 'Liverpool_Ion_Wavenet_(1)_(1)_(1).ipynb', 'Liverpool_Ion_Wavenet_(1).ipynb', 'Liverpool_Ion_Wavenet_(1)_(1)_(1) (1).ipynb', 'lgb_model (12).ipynb', 'lgb_model (10) (2).ipynb', 'lgb_model (10) (1).ipynb', 'lgb_model (10).ipynb', 'Liverpool_Ion_fastai_(1)_(1)_(1)_(1)_(1).ipynb', 'sales_train_CA_1', 'sales_train_CA_2', 'sales_train_CA_3', 'sales_train_CA_4', 'sales_train_TX_1', 's

In [0]:
path = '../content/drive/My Drive/Colab Notebooks/'

In [0]:
#reduce memory
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

**DATA PREPARATION**

In [0]:
# sales_train_validation = pd.read_csv(path + 'sales_train_validation.csv')
sell_price = pd.read_csv(path + 'sell_prices.csv')
calendar = pd.read_csv(path + 'calendar.csv')

In [0]:
sales_train_validation = pd.melt(sales_train_validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
sales_train_validation['day'] = sales_train_validation['day'].apply(lambda x: int(x[2:])).astype('int16')
# sales_train_validation = sales_train_validation.loc[sales_train_validation.day >= 1069] #day 1434 is 01/01/2015 1069, 949

In [0]:
sales_train_CA_1 = sales_train_validation.loc[sales_train_validation['store_id']=='CA_1']
sales_train_CA_2 = sales_train_validation.loc[sales_train_validation['store_id']=='CA_2']
sales_train_CA_3 = sales_train_validation.loc[sales_train_validation['store_id']=='CA_3']
sales_train_CA_4 = sales_train_validation.loc[sales_train_validation['store_id']=='CA_4']
sales_train_TX_1 = sales_train_validation.loc[sales_train_validation['store_id']=='TX_1']
sales_train_TX_2 = sales_train_validation.loc[sales_train_validation['store_id']=='TX_2']
sales_train_TX_3 = sales_train_validation.loc[sales_train_validation['store_id']=='TX_3']
sales_train_WI_1 = sales_train_validation.loc[sales_train_validation['store_id']=='WI_1']
sales_train_WI_2 = sales_train_validation.loc[sales_train_validation['store_id']=='WI_2']
sales_train_WI_3 = sales_train_validation.loc[sales_train_validation['store_id']=='WI_3']

In [0]:
del sales_train_validation
gc.collect()

0

In [0]:
gc.collect()

0

In [0]:
calendar['d'] = calendar['d'].apply(lambda x: int(x[2:])).astype('int16')

In [0]:
calendar.drop(['weekday'], axis=1, inplace=True)

In [0]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [0]:
add_datepart(calendar, 'date', drop=False)

In [0]:
def to_season(date):
    
    #convert date to season for the northen hemisphere
    #"day of year" ranges for the northern hemisphere
    spring = range(80, 172)
    summer = range(172, 264)
    fall = range(264, 355)
    # winter = everything else
    doy = date.timetuple().tm_yday
    if doy in spring:
      season = 'Spring'
    elif doy in summer:
      season = 'Summer'
    elif doy in fall:
      season = 'Fall'
    else:
      season = 'Winter'
    return season

In [0]:
calendar['Season'] = calendar.date.transform(lambda x: to_season(x))
# calendar.drop(['Day'], axis=1, inplace=True)

In [0]:
sell_price['price_max'] = sell_price.groupby(['store_id','item_id'])['sell_price'].transform('max')
sell_price['price_min'] = sell_price.groupby(['store_id','item_id'])['sell_price'].transform('min')
sell_price['price_std'] = sell_price.groupby(['store_id','item_id'])['sell_price'].transform('std')
sell_price['price_mean'] = sell_price.groupby(['store_id','item_id'])['sell_price'].transform('mean')
sell_price['price_norm'] = sell_price['sell_price']/sell_price['price_max']
sell_price['price_nunique'] = sell_price.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
sell_price['item_nunique'] = sell_price.groupby(['store_id','sell_price'])['item_id'].transform('nunique')
calendar_prices = calendar[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
sell_price = sell_price.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices
gc.collect()
sell_price['price_momentum'] = sell_price['sell_price']/sell_price.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
sell_price['price_momentum_m'] = sell_price['sell_price']/sell_price.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
sell_price['price_momentum_y'] = sell_price['sell_price']/sell_price.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')
del sell_price['month'], sell_price['year']
gc.collect()

0

In [0]:
#merge calendar with sales_train_validation
sales_train_CA_1 = sales_train_CA_1.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_CA_1.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_CA_1 = sales_train_CA_1.merge(sell_price, how='left', on=['item_id','store_id','wm_yr_wk'])
sales_train_CA_1.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_CA_2 = sales_train_CA_2.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_CA_2.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_CA_2 = sales_train_CA_2.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_CA_2.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_CA_3 = sales_train_CA_3.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_CA_3.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_CA_3 = sales_train_CA_3.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_CA_3.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_CA_4 = sales_train_CA_4.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_CA_4.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_CA_4 = sales_train_CA_4.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_CA_4.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_TX_1 = sales_train_TX_1.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_TX_1.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_TX_1 = sales_train_TX_1.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_TX_1.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_TX_2 = sales_train_TX_2.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_TX_2.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_TX_2 = sales_train_TX_2.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_TX_2.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_TX_3 = sales_train_TX_3.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_TX_3.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_TX_3 = sales_train_TX_3.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_TX_3.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_WI_1 = sales_train_WI_1.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_WI_1.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_WI_1 = sales_train_WI_1.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_WI_1.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_WI_2 = sales_train_WI_2.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_WI_2.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_WI_2 = sales_train_WI_2.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_WI_2.drop(['store_id','state_id'], axis=1, inplace=True)

#merge calendar with sales_train_validation
sales_train_WI_3 = sales_train_WI_3.merge(calendar, how='left', left_on='day', right_on='d')
sales_train_WI_3.drop('d', axis=1, inplace=True)
#merge sell_price with sales_train_validation
sales_train_WI_3 = sales_train_WI_3.merge(sell_price, how='left', on=['item_id','store_id', 'wm_yr_wk'])
sales_train_WI_3.drop(['store_id','state_id'], axis=1, inplace=True)

In [0]:
data_set = [sales_train_CA_1,sales_train_CA_2,sales_train_CA_3,sales_train_CA_4,sales_train_TX_1,
            sales_train_TX_2,sales_train_TX_3,sales_train_WI_1,sales_train_WI_2,sales_train_WI_3]

names = ['sales_train_CA_1','sales_train_CA_2','sales_train_CA_3','sales_train_CA_4','sales_train_TX_1',
         'sales_train_TX_2','sales_train_TX_3','sales_train_WI_1','sales_train_WI_2','sales_train_WI_3']

for data,name in zip(data_set,names):
  data = data.reset_index(drop=True)
  data.to_feather(name)

!cp '/content/sales_train_CA_1' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_CA_2' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_CA_3' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_CA_4' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_TX_1' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_TX_2' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_TX_3' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_WI_1' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_WI_2' '../content/drive/My Drive/Colab Notebooks/'
!cp '/content/sales_train_WI_3' '../content/drive/My Drive/Colab Notebooks/'

**FEATURE ENGINEERING**

In [0]:
sales_train_CA_1 = pd.read_feather(path + 'sales_train_CA_1')
# sales_train_CA_2 = pd.read_feather(path + 'sales_train_CA_2')
# sales_train_CA_3 = pd.read_feather(path + 'sales_train_CA_3')
# sales_train_CA_4 = pd.read_feather('sales_train_CA_4')
# sales_train_TX_1 = pd.read_feather('sales_train_TX_1')
# sales_train_TX_2 = pd.read_feather('sales_train_TX_2')
# sales_train_TX_3 = pd.read_feather(path + 'sales_train_TX_3')
# sales_train_WI_1 = pd.read_feather('sales_train_WI_1')
# sales_train_WI_2 = pd.read_feather('sales_train_WI_2')
# sales_train_WI_3 = pd.read_feather('sales_train_WI_3')

In [0]:
# def fillna_price(data):
#   mean_sell_price = data.loc[data.sell_price.isnull()==False][['item_id','sell_price']].groupby('item_id', as_index=False).mean()
#   mean_sell_price_per_item = {}
#   for i in range(mean_sell_price.shape[0]):
#     mean_sell_price_per_item[mean_sell_price['item_id'].iloc[i]] = mean_sell_price['sell_price'].iloc[i]
    
#   sell_price_nan = data['sell_price'].isnull().values
#   sell_price_nan_loc = np.where(sell_price_nan == True)[0]

#   for i in sell_price_nan_loc:
#     id_ = data['item_id'].iloc[i]
#     data['sell_price'].iloc[i] = mean_sell_price_per_item[id_]

#   return data

In [0]:
def label_encode(data):

  cat_col = ['item_id', 'dept_id', 'cat_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'Season']
  for col in cat_col:
    data[col] = data[col].astype('category').cat.codes + 1

  return data

In [0]:
def create_fe(data):
    
    shift_window_size = [1]
    lag_day = [1,3,5,7,28]
    roll_demand_days = [7,14,30,60,180]
    # expand_demand_days = [1,7]
    EWF_windows = [7,14,30,60]
    roll_price_days = [7,14,30,60]


    #### Add lag demand feature     
    for i in lag_day:
        data['lag_pos'+str(i)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i))
        # data['lag_neg'+str(i)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i))

    ###Add price change
    gr = data.groupby(["item_id"])["sell_price"]
    data["sell_price_rel_diff"] = gr.pct_change()
    data["sell_price_cumrel"] = (gr.shift(0) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    data["sell_price_cumrel"] = (gr.shift(7) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    data["sell_price_cumrel"] = (gr.shift(14) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    data["sell_price_cumrel"] = (gr.shift(30) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())    
    

    ### Moving average on item_id         
    for i in shift_window_size:
      for j in roll_demand_days:
        data['demand_roll_mean_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).mean())
        data['demand_roll_std_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).std())
        # data['demand_roll_q10_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.10))
        data['demand_roll_q25_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.25))
        # data['demand_roll_q50_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.50))
        data['demand_roll_q75_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.75))
        # data['demand_roll_q90_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(window=j, min_periods=1).quantile(0.90))
        data['demand_roll_IQR_'+str(j)] = data['demand_roll_q75_'+str(j)] - data['demand_roll_q25_'+str(j)]    
        
    # for i in shift_window_size:
    #   for j in expand_demand_days:
    #     data['demand_expand_mean_'+str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).expanding(j).mean())    


    # Add exponential weighted functions:
    for i in shift_window_size:
      for j in EWF_windows:
        data['ewm_mean_' + str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).ewm(span=j, min_periods=1).mean())
        data['ewm_std_' + str(j)] = data.groupby('item_id', as_index=False)['demand'].transform(lambda x: x.shift(i).ewm(span=j, min_periods=1).std())

    for i in roll_price_days:
      data['price_roll_mean_'+str(i)] = data.groupby('item_id', as_index=False)['sell_price'].transform(lambda x: x.rolling(i).mean())


    ### Moving average on item_id and cat_id
    # for i in shift_window_size:
    #    for j in roll_demand_days:
    #      data['demand_cat_roll_mean_'+str(j)] = data.groupby(['item_id','cat_id'], as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(j).mean())    

    # for i in shift_window_size:
    #   for j in expand_demand_days:
    #     data['demand_cat_expand_mean_'+str(j)] = data.groupby(['item_id','cat_id'], as_index=False)['demand'].transform(lambda x: x.shift(i).expanding(j).mean())       

    # for i in roll_price_days:
    #   data['price_cat_roll_mean_'+str(i)] = data.groupby(['item_id','cat_id'], as_index=False)['sell_price'].transform(lambda x: x.rolling(i).mean())


    ### Moving average on item_id and dept_id
    for i in shift_window_size:
       for j in roll_demand_days:
         data['demand_dept_roll_mean_'+str(j)] = data.groupby(['item_id','dept_id'], as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(j).mean())    

    # for i in shift_window_size:
    #   for j in expand_demand_days:
    #     data['demand_dept_expand_mean_'+str(j)] = data.groupby(['item_id','dept_id'], as_index=False)['demand'].transform(lambda x: x.shift(i).expanding(j).mean())       

    # for i in roll_price_days:
    #   data['price_dept_roll_mean_'+str(i)] = data.groupby(['item_id','dept_id'], as_index=False)['sell_price'].transform(lambda x: x.rolling(i).mean())

    ### Moving average on item_id and Season
    # for i in shift_window_size:
    #    for j in roll_demand_days:
    #      data['demand_sea_roll_mean_'+str(j)] = data.groupby(['item_id','Season'], as_index=False)['demand'].transform(lambda x: x.shift(i).rolling(j).mean())    

    # for i in shift_window_size:
    #   for j in expand_demand_days:
    #     data['demand_sea_expand_mean_'+str(j)] = data.groupby(['item_id','Season'], as_index=False)['demand'].transform(lambda x: x.shift(i).expanding(j).mean())       

    # for i in roll_price_days:
    #   data['price_sea_roll_mean_'+str(i)] = data.groupby(['item_id','Season'], as_index=False)['sell_price'].transform(lambda x: x.rolling(i).mean())

    data['item_id_cat_id_mean'] = data.groupby(['item_id','cat_id'], as_index=False)['demand'].transform(lambda x: x.shift(1).mean())
    data['item_id_cat_id_std'] = data.groupby(['item_id','cat_id'], as_index=False)['demand'].transform(lambda x: x.shift(1).std())
    data['item_id_dept_id_mean'] = data.groupby(['item_id','dept_id'], as_index=False)['demand'].transform(lambda x: x.shift(1).mean())
    data['item_id_dept_id_std'] = data.groupby(['item_id','dept_id'], as_index=False)['demand'].transform(lambda x: x.shift(1).std())
    data['dept_id_cat_id_mean'] = data.groupby(['dept_id','cat_id'], as_index=False)['demand'].transform(lambda x: x.shift(1).mean())
    data['dept_id_cat_id_std'] = data.groupby(['dept_id','cat_id'], as_index=False)['demand'].transform(lambda x: x.shift(1).std())

    return data

In [0]:
def find_last_sale(df,n_day):
    
    # Limit initial df
    ls_df = df[['item_id','day','demand']]
    
    # Convert target to binary
    ls_df['non_zero'] = (ls_df['demand']>0).astype(np.int8)
    
    # Make lags to prevent any leakage
    ls_df['non_zero_lag'] = ls_df.groupby(['item_id'])['non_zero'].transform(lambda x: x.shift(n_day).rolling(2000,1).sum()).fillna(-1)

    temp_df = ls_df[['item_id','day','non_zero_lag']].drop_duplicates(subset=['item_id','non_zero_lag'])
    temp_df.columns = ['item_id','day_min','non_zero_lag']

    ls_df = ls_df.merge(temp_df, on=['item_id','non_zero_lag'], how='left')
    ls_df['last_sale'] = ls_df['day'] - ls_df['day_min']
    df['last_sale'] = ls_df['last_sale'].values

    return df

In [0]:
#prepare submission file
sample_submission = pd.read_csv(path+'sample_submission.csv')
sales_train_validation1 = pd.read_csv(path + 'sales_train_validation.csv')
sales_train_validation1 = sales_train_validation1[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']]
validation = sample_submission.loc[sample_submission.id.str.contains('validation')]
validation = validation.merge(sales_train_validation1, how='left', on='id')
validation = pd.melt(validation, id_vars = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name = 'day', value_name = 'demand')
validation['day'] = validation['day'].apply(lambda x: int(x[1:])).astype('int16')
validation['day'] = validation['day'] + 1913

In [0]:
validation_CA_1 = validation.loc[validation['store_id']=='CA_1']
validation_CA_2 = validation.loc[validation['store_id']=='CA_2']
validation_CA_3 = validation.loc[validation['store_id']=='CA_3']
validation_CA_4 = validation.loc[validation['store_id']=='CA_4']
validation_TX_1 = validation.loc[validation['store_id']=='TX_1']
validation_TX_2 = validation.loc[validation['store_id']=='TX_2']
validation_TX_3 = validation.loc[validation['store_id']=='TX_3']
validation_WI_1 = validation.loc[validation['store_id']=='WI_1']
validation_WI_2 = validation.loc[validation['store_id']=='WI_2']
validation_WI_3 = validation.loc[validation['store_id']=='WI_3']

In [0]:
CA_1_submission = pd.DataFrame({'id': validation_CA_1['id'], 'day':validation_CA_1['day'], 'demand':0})
CA_2_submission = pd.DataFrame({'id': validation_CA_2['id'], 'day':validation_CA_2['day'], 'demand':0})
CA_3_submission = pd.DataFrame({'id': validation_CA_3['id'], 'day':validation_CA_3['day'], 'demand':0})
CA_4_submission = pd.DataFrame({'id': validation_CA_4['id'], 'day':validation_CA_4['day'], 'demand':0})
TX_1_submission = pd.DataFrame({'id': validation_TX_1['id'], 'day':validation_TX_1['day'], 'demand':0})
TX_2_submission = pd.DataFrame({'id': validation_TX_2['id'], 'day':validation_TX_2['day'], 'demand':0})
TX_3_submission = pd.DataFrame({'id': validation_TX_3['id'], 'day':validation_TX_3['day'], 'demand':0})
WI_1_submission = pd.DataFrame({'id': validation_WI_1['id'], 'day':validation_WI_1['day'], 'demand':0})
WI_2_submission = pd.DataFrame({'id': validation_WI_2['id'], 'day':validation_WI_2['day'], 'demand':0})
WI_3_submission = pd.DataFrame({'id': validation_WI_3['id'], 'day':validation_WI_3['day'], 'demand':0})

In [0]:
# from xgboost import XGBRegressor as xgb
import lightgbm as lgb

In [0]:
# train = [sales_train_CA_1,sales_train_CA_2,sales_train_CA_3,sales_train_CA_4,sales_train_TX_1,
#             sales_train_TX_2,sales_train_TX_3,sales_train_WI_1,sales_train_WI_2,sales_train_WI_3]

# test = [validation_CA_1,validation_CA_2,validation_CA_3,validation_CA_4,validation_TX_1,
#         validation_TX_2,validation_TX_3,validation_WI_1,validation_WI_2,validation_WI_3]

# submission = [CA_1_submission,CA_2_submission,CA_3_submission,CA_4_submission,TX_1_submission,
#               TX_2_submission,TX_3_submission,WI_1_submission,WI_2_submission,WI_3_submission]

train = [sales_train_CA_1]
test = [validation_CA_1]
submission = [CA_1_submission]
sub_name = ['CA_1_submission.csv']

lgb_params = {      'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

In [0]:
sales_train_CA_1.drop(['Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'], axis=1, inplace=True)

In [0]:
sales_train_CA_1 = reduce_mem_usage(sales_train_CA_1)

Mem. usage decreased to 667.50 Mb (54.9% reduction)


In [0]:
gc.collect()

0

In [0]:
for df_train1,df_test,df_sub,name in zip(train, test, submission, sub_name):    
  df_train = df_train1.copy()
  df_train.drop(['id','date','wm_yr_wk','Dayofweek'], axis=1, inplace=True)
  gc.collect()
  df_train = label_encode(df_train)
  print('Doing feature engineering')
  # df_train = find_last_sale(df_train,1)
  df_train = create_fe(df_train)  
  df_train = df_train.fillna(-1) 
  
  X = df_train.loc[df_train['day']<1858].drop('demand', axis=1)
  y = df_train.loc[df_train['day']<1858]['demand']
  Xval = df_train.loc[df_train['day']>=1858].drop('demand', axis=1)
  yval = df_train.loc[df_train['day']>=1858]['demand']

  # #add PCA feature
  # pca = PCA(n_components = 4)
  # pca.fit(X)
  # pca_X = pd.DataFrame(pca.transform(X))
  # pca_X.columns = ['pca1', 'pca2', 'pca3', 'pca4']

  # for col in pca_X.columns:
  #   X[col] = pca_X[col].values

  # pca_Xval = pd.DataFrame(pca.transform(Xval))
  # pca_Xval.columns = ['pca1', 'pca2', 'pca3', 'pca4']

  # for col in pca_Xval.columns:
  #   Xval[col] = pca_Xval[col].values
  
  train_data = lgb.Dataset(X, label=y)
  val_data = lgb.Dataset(Xval, label=yval)

  del df_train, X, y, Xval, yval
  gc.collect()

  print('Running LGB')
  lgb_model = lgb.train(lgb_params, train_data, valid_sets=[train_data, val_data], verbose_eval=10)

  #merge calendar with test
  df_test = df_test.merge(calendar, how='left', left_on='day', right_on='d')
  df_test.drop('d', axis=1, inplace=True)
  #merge sell_price with test
  df_test = df_test.merge(sell_price, how='left', on=['item_id','store_id','wm_yr_wk'])
  df_test.drop(['store_id','state_id'], axis=1, inplace=True)

  df = pd.concat([df_train1,df_test])
  df.drop(['id','date','wm_yr_wk','Dayofweek'], axis=1, inplace=True)
  df = label_encode(df)
  # preds = np.zeros(df_sub.shape[0])
  print('Doing prediction')
  for i in range(28):
    print('i = ',i)
    day = 1913 + i + 1
    temp = df.loc[df['day']<=day]
    # temp = find_last_sale(temp,1)
    temp = create_fe(temp)
    temp = temp.fillna(-1)
    temp.drop(['demand','Is_month_end', 'Is_month_start', 
               'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start', 'Elapsed'], axis=1, inplace=True)
    # pca_test = pd.DataFrame(pca.transform(temp))
    # pca_test.columns = ['pca1', 'pca2', 'pca3', 'pca4']

    # for col in pca_test.columns:
    #   temp[col] = pca_test[col].values
    
    
    temp = temp.loc[temp['day'] == day]
    # print(temp.shape)
    temp['demand'] = lgb_model.predict(temp)
    # print(df_sub.loc[df_sub['day']==day].shape)
    df_sub.loc[df_sub['day']==day, 'demand'] = temp['demand'].values
    df.loc[df['day']==day, 'demand'] = temp['demand'].values
    del temp
    gc.collect()    
  
  df_sub.to_csv(name, index=False)

Doing feature engineering
Running LGB
[10]	training's rmse: 3.69851	valid_1's rmse: 3.11618
[20]	training's rmse: 3.27072	valid_1's rmse: 2.74092
[30]	training's rmse: 2.93199	valid_1's rmse: 2.45665
[40]	training's rmse: 2.69677	valid_1's rmse: 2.26661
[50]	training's rmse: 2.54316	valid_1's rmse: 2.14896
[60]	training's rmse: 2.44339	valid_1's rmse: 2.0769
[70]	training's rmse: 2.37861	valid_1's rmse: 2.03475
[80]	training's rmse: 2.33789	valid_1's rmse: 2.01126
[90]	training's rmse: 2.30896	valid_1's rmse: 1.99726
[100]	training's rmse: 2.28855	valid_1's rmse: 1.9887
[110]	training's rmse: 2.27437	valid_1's rmse: 1.98511
[120]	training's rmse: 2.26208	valid_1's rmse: 1.98229
[130]	training's rmse: 2.25215	valid_1's rmse: 1.9809
[140]	training's rmse: 2.2448	valid_1's rmse: 1.98009
[150]	training's rmse: 2.23741	valid_1's rmse: 1.97936
[160]	training's rmse: 2.23071	valid_1's rmse: 1.97892
[170]	training's rmse: 2.22544	valid_1's rmse: 1.97913
[180]	training's rmse: 2.22025	valid_1's

In [0]:
!cp '/content/CA_1_submission.csv' '../content/drive/My Drive/Colab Notebooks/'

**SUBMISSION OF 10 SMALL STORE SPLIT MODELS**

In [0]:
path

'../content/drive/My Drive/Colab Notebooks/'

In [0]:
sample_submission = pd.read_csv(path+'sample_submission.csv')

In [0]:
CA_1_submission = pd.read_csv(path+'CA_1_submission.csv')
CA_2_submission = pd.read_csv(path+'CA_2_submission.csv')
CA_3_submission = pd.read_csv(path+'CA_3_submission.csv')
CA_4_submission = pd.read_csv(path+'CA_4_submission.csv')
TX_1_submission = pd.read_csv(path+'TX_1_submission.csv')
TX_2_submission = pd.read_csv(path+'TX_2_submission.csv')
TX_3_submission = pd.read_csv(path+'TX_3_submission.csv')
WI_1_submission = pd.read_csv(path+'WI_1_submission.csv')
WI_2_submission = pd.read_csv(path+'WI_2_submission.csv')
WI_3_submission = pd.read_csv(path+'WI_3_submission.csv')

In [0]:
sample_submission = sample_submission[:30490]

In [0]:
sample = pd.melt(sample_submission, id_vars = ['id'], var_name = 'day', value_name = 'demand')

In [0]:
sample['day'] = sample['day'].apply(lambda x: int(x[1:])).astype('int16')

In [0]:
sample.drop('demand', axis=1, inplace=True)

In [0]:
sample['day'] = sample['day'] + 1913

In [0]:
result = pd.concat([CA_1_submission,CA_2_submission,CA_3_submission,CA_4_submission,
                    TX_1_submission,TX_2_submission,TX_3_submission,
                    WI_1_submission,WI_2_submission,WI_3_submission])

In [0]:
sample = sample.merge(result, how='left', on=['id','day'])

In [0]:
validation_submission = pd.DataFrame({'id':sample_submission['id']})

In [0]:
for i in range(0,28):
    validation_submission['F'+str(i+1)] = np.array(sample['demand'][(30490*i):(30490*(i+1))])

In [0]:
evaluation_submission = validation_submission.copy()

In [0]:
submission = pd.concat([validation_submission, evaluation_submission])

In [0]:
submission['id'] = sample_submission['id'].values

In [0]:
submission.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,6.098000e+04,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,0.001343,0.001387,0.001423,0.001296,0.001282,0.001252,0.001077,0.000750,0.000937,0.000662,0.000841,0.000811,0.000627,0.000442,0.000442,0.000476,0.000498,0.000587,0.000498,0.000282,0.000288,0.000233,2.942054e-04,0.000257,0.000317,0.000307,0.000151,0.000157
std,0.005848,0.004117,0.003994,0.003292,0.003244,0.002934,0.002395,0.002062,0.002737,0.001680,0.002261,0.002235,0.002207,0.001159,0.001325,0.001380,0.001463,0.001720,0.001472,0.000778,0.000868,0.000670,9.371687e-04,0.000745,0.000860,0.000802,0.000389,0.000397
min,0.000001,0.000005,0.000011,0.000010,0.000005,0.000007,0.000006,0.000004,0.000002,0.000004,0.000004,0.000006,0.000005,0.000005,0.000003,0.000002,0.000002,0.000002,0.000007,0.000003,0.000003,0.000002,7.644059e-07,0.000002,0.000007,0.000006,0.000005,0.000002
25%,0.000141,0.000187,0.000191,0.000186,0.000222,0.000258,0.000195,0.000121,0.000137,0.000113,0.000134,0.000142,0.000107,0.000090,0.000073,0.000076,0.000069,0.000076,0.000081,0.000061,0.000056,0.000044,4.629421e-05,0.000050,0.000064,0.000063,0.000038,0.000037
50%,0.000336,0.000421,0.000426,0.000416,0.000476,0.000553,0.000448,0.000284,0.000305,0.000261,0.000295,0.000313,0.000230,0.000192,0.000156,0.000164,0.000153,0.000156,0.000159,0.000111,0.000103,0.000079,8.865837e-05,0.000088,0.000108,0.000112,0.000063,0.000063
75%,0.000852,0.001023,0.001065,0.001030,0.001080,0.001158,0.001036,0.000620,0.000716,0.000550,0.000684,0.000683,0.000523,0.000399,0.000369,0.000368,0.000379,0.000390,0.000368,0.000234,0.000220,0.000165,1.879715e-04,0.000180,0.000209,0.000226,0.000119,0.000135
max,0.315974,0.159242,0.155664,0.133986,0.143396,0.146881,0.096268,0.087172,0.109043,0.048114,0.084744,0.086075,0.118897,0.041372,0.044372,0.040000,0.040200,0.051470,0.048016,0.026782,0.029544,0.029911,4.905097e-02,0.033156,0.020319,0.021882,0.013637,0.012176


In [0]:
submission.to_csv('submission.csv', index=False)

In [0]:
submission.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,1.220020,1.178965,1.187434,1.201349,1.363002,1.693079,1.751441,1.484504,1.544116,1.340862,1.352255,1.539889,1.831907,1.607268,1.550285,1.362289,1.417775,1.438198,1.486356,1.841886,1.967124,1.446200,1.303840,1.300211,1.310740,1.444938,1.739248,1.743915
std,2.605930,2.431350,2.408229,2.473661,2.841509,3.535596,3.573507,3.054047,3.165467,2.693780,2.877897,3.302647,3.839855,3.551854,3.196482,2.759724,2.945000,3.095184,3.143542,3.950460,4.042510,2.895552,2.659793,2.653828,2.762982,3.075282,3.732135,3.631683
min,0.000721,0.020110,0.026659,0.024205,0.029232,0.043205,0.042513,0.032829,0.038529,0.040558,0.030565,0.037885,0.045078,0.038823,0.030087,0.033055,0.030520,0.029548,0.030291,0.045561,0.042882,0.026792,0.025260,0.024416,0.021645,0.026220,0.033358,0.030822
25%,0.231287,0.269086,0.276550,0.276729,0.304187,0.365782,0.384642,0.307110,0.318202,0.298164,0.290812,0.318948,0.382608,0.329596,0.317138,0.287217,0.301069,0.302486,0.319118,0.378344,0.414779,0.313004,0.282955,0.296015,0.294855,0.320892,0.372191,0.377205
50%,0.552980,0.577069,0.579741,0.570485,0.635935,0.784234,0.831820,0.672070,0.705153,0.630669,0.607807,0.682318,0.816433,0.719988,0.706257,0.633962,0.649110,0.639482,0.672079,0.824620,0.907616,0.682582,0.615004,0.609010,0.594119,0.654474,0.772216,0.797245
75%,1.225079,1.177540,1.184204,1.198681,1.325668,1.693087,1.798016,1.459515,1.535732,1.341790,1.327057,1.488200,1.806311,1.564601,1.544173,1.360273,1.405892,1.395937,1.445931,1.778399,1.971997,1.448918,1.289089,1.287587,1.284108,1.392385,1.684112,1.715495
max,103.129826,96.196833,97.743190,100.777007,113.170946,134.769702,125.204108,102.611476,97.578504,97.761674,105.434750,116.261249,138.048988,122.569355,108.121420,101.152655,100.683121,105.011088,116.029930,134.740988,125.325292,101.584592,94.177146,96.342706,102.686281,116.448463,137.618544,127.231507


In [0]:
submission.columns

Index(['id', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [0]:
cols = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10',
       'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28']

In [0]:
for col in cols:
  submission[col] = submission[col].apply(lambda x: 0 if x<0.5 else x)

In [0]:
submission.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,1.113775,1.072239,1.077842,1.092042,1.263611,1.607028,1.656270,1.385646,1.431175,1.224207,1.227463,1.435791,1.743918,1.504226,1.434396,1.242399,1.300957,1.328620,1.390999,1.744186,1.851851,1.324698,1.187815,1.174183,1.187979,1.328497,1.646632,1.645973
std,2.677465,2.530232,2.496205,2.553157,2.902368,3.565083,3.602027,3.117032,3.218199,2.778813,2.921101,3.331401,3.843556,3.567087,3.210153,2.809466,2.983006,3.130765,3.179818,3.939025,3.979390,2.916455,2.728314,2.703921,2.798620,3.079382,3.719005,3.628785
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.551543,0.583428,0.582357,0.577109,0.645202,0.799287,0.822870,0.680506,0.700662,0.626239,0.607003,0.688008,0.827629,0.720991,0.704731,0.634334,0.648662,0.645336,0.685096,0.823271,0.905526,0.683239,0.618087,0.609214,0.601886,0.664435,0.786698,0.806761
75%,1.232778,1.188356,1.191913,1.207932,1.349314,1.708085,1.770354,1.467369,1.514916,1.330381,1.303967,1.485556,1.810879,1.556598,1.537444,1.349916,1.393946,1.401064,1.458280,1.778337,1.949259,1.450420,1.295698,1.285834,1.283775,1.395995,1.696020,1.729692
max,105.970309,101.830985,102.315847,104.094949,111.453359,126.662387,127.865020,108.582037,104.488580,104.744448,107.733045,114.243882,131.683670,124.140021,110.014600,105.301022,105.486680,107.173268,113.778271,130.160282,125.548569,107.243963,102.171130,103.051036,105.131592,112.788379,130.027923,125.675120


In [0]:
submission.to_csv('submission1.csv', index=False)